In [1]:
import pandas as pd
train=pd.read_csv("/Users/shichangtai/Desktop/Kaggle/Bag of Wprds Meet Bags of Popcorn/labeledTrainData.tsv",header=0,delimiter='\t',quoting=3)
test=pd.read_csv("/Users/shichangtai/Desktop/Kaggle/Bag of Wprds Meet Bags of Popcorn/testData.tsv",header=0,delimiter='\t',quoting=3)
unlabeled_train=pd.read_csv("/Users/shichangtai/Desktop/Kaggle/Bag of Wprds Meet Bags of Popcorn/unlabeledTrainData.tsv",header=0,delimiter='\t',quoting=3)
print("Load %d labeled train data, %d unlabeled train data and %d test data" % (train["review"].size, unlabeled_train["review"].size, len(test['review'])))

Load 25000 labeled train data, 50000 unlabeled train data and 25000 test data


In [2]:
from bs4 import BeautifulSoup
import re
import nltk
from nltk.corpus import stopwords
def review_to_words(raw_review,remove_stopwords=False):
    #1. remove html
    raw_text=BeautifulSoup(raw_review,'lxml').get_text()
    #2. remove non-letter, replace n't (maybe we can hold the number)
    only_letter_text=re.sub("\d","NUM",raw_text)
    only_letter_text=re.sub("n\'t"," not",only_letter_text)
    only_letter_text=re.sub("[^a-zA-Z]"," ",only_letter_text)
    #3. split and lower case
    pure_text=only_letter_text.lower().split()
    #4. (optional) remove stop words
    if remove_stopwords:
        stemmer=nltk.PorterStemmer()
        stopword=set(stopwords.words("english"))
        pure_text=[stemmer(m) for m in pure_text if m not in stopword]
    #5. return a list of words
    return (" ".join(pure_text))

In [3]:
tokenizer=nltk.data.load('tokenizers/punkt/english.pickle')
def review_to_sentences(review,tokenizer,remove_stopwords=False):
    sentences=[]
    raw_sentences=tokenizer.tokenize(review.strip())
    for item in raw_sentences:
        if(len(item)>0):
            sentences.append(review_to_words(item,remove_stopwords))
    return sentences

In [4]:
num_reviews=len(train["review"])
print("Cleaning and parsing the training set movie reviews...\n")
clean_train_reviews=[]
for x in xrange(0,num_reviews):
    if((x+1)%1000==0):
        print ("Review %d of %d\n" % (x+1,num_reviews))
    clean_train_reviews.append(review_to_words(train["review"][x]))

num_reviews=len(unlabeled_train["review"])
print("Cleaning and parsing the unlabeled_train set movie reviews...\n")
unlabeled_clean_train_reviews=[]
for x in xrange(0,num_reviews):
    if((x+1)%1000==0):
        print ("Review %d of %d\n" % (x+1,num_reviews))
    unlabeled_clean_train_reviews.append(review_to_words(unlabeled_train["review"][x]))
    
num_reviews=len(test["review"])
print("Cleaning and parsing the test set movie reviews...\n")
clean_test_reviews=[]
for x in xrange(0,num_reviews):
    if((x+1)%1000==0):
        print ("Review %d of %d\n" % (x+1,num_reviews))
    clean_test_reviews.append(review_to_words(test["review"][x]))

Cleaning and parsing the training set movie reviews...

Review 1000 of 25000

Review 2000 of 25000

Review 3000 of 25000

Review 4000 of 25000

Review 5000 of 25000

Review 6000 of 25000

Review 7000 of 25000

Review 8000 of 25000

Review 9000 of 25000

Review 10000 of 25000

Review 11000 of 25000

Review 12000 of 25000

Review 13000 of 25000

Review 14000 of 25000

Review 15000 of 25000

Review 16000 of 25000

Review 17000 of 25000

Review 18000 of 25000

Review 19000 of 25000

Review 20000 of 25000

Review 21000 of 25000

Review 22000 of 25000

Review 23000 of 25000

Review 24000 of 25000

Review 25000 of 25000

Cleaning and parsing the unlabeled_train set movie reviews...

Review 1000 of 50000

Review 2000 of 50000

Review 3000 of 50000

Review 4000 of 50000

Review 5000 of 50000

Review 6000 of 50000

Review 7000 of 50000

Review 8000 of 50000

Review 9000 of 50000

Review 10000 of 50000

Review 11000 of 50000

Review 12000 of 50000

Review 13000 of 50000

Review 14000 of 50000

Re

In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import SGDClassifier

print "Vectorizing..."

vectorizer = TfidfVectorizer( min_df=2, max_df=0.95, max_features = 50000, ngram_range = ( 1, 4 ),
                              sublinear_tf = True )

vectorizer = vectorizer.fit(clean_train_reviews + unlabeled_clean_train_reviews)
train_data_features = vectorizer.transform( clean_train_reviews )
test_data_features = vectorizer.transform( clean_test_reviews )

#print "Reducing dimension..."

#from sklearn.feature_selection.univariate_selection import SelectKBest, chi2, f_classif
#fselect = SelectKBest(chi2 , k=40000)
#train_data_features = fselect.fit_transform(train_data_features, train["sentiment"])
#test_data_features = fselect.transform(test_data_features)

print "Training..."

model1 = MultinomialNB(alpha=0.0005)
model1.fit( train_data_features, train["sentiment"] )

model2 = SGDClassifier(loss='modified_huber', n_iter=5, random_state=0, shuffle=True)
model2.fit( train_data_features, train["sentiment"] )

p1 = model1.predict_proba( test_data_features )[:,1]
p2 = model2.predict_proba( test_data_features )[:,1]

print "Writing results..."

output = pd.DataFrame( data = { "id": test["id"], "sentiment": .2*p1 + 1.*p2 } )
output.to_csv("/Users/shichangtai/Desktop/Kaggle/Bag of Wprds Meet Bags of Popcorn/result_Combine_BagOfWords.csv",index=False,quoting=3)

Vectorizing...
Training...
Writing results...


In [ ]:
# amazing 0.96699!